In [ ]:
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, DallETool


# ────────────────────────────────────────────────────────────────────────────────
# Variables - these can be changed:
# ────────────────────────────────────────────────────────────────────────────────

location = "Whitstable" # This value can be changed to any town or city in the UK.

verbose_agents = True # A value of `True` (recommended) will instruct the Agent to output information while it is in progress; or `False` to get no information and just the result.


# ────────────────────────────────────────────────────────────────────────────────
# Initialize constants - please don't change these values:
# ────────────────────────────────────────────────────────────────────────────────

load_dotenv()

scrape_website_tool = ScrapeWebsiteTool()

local_events_url = "https://www.wherecanwego.com/whats-on/{{location}}"
geocoding_url = "https://geocoding-api.open-meteo.com/v1/search?name={{location}}"
weather_url = "https://api.open-meteo.com/v1/forecast?latitude={{latitude}}&longitude={{longitude}}&daily=cloud_cover_mean,temperature_2m_mean,rain_sum,wind_speed_10m_mean&forecast_days=1&format=json&timeformat=unixtime"


# ────────────────────────────────────────────────────────────────────────────────
# Build the agents
# ────────────────────────────────────────────────────────────────────────────────
geocoder = Agent(
    role="Geocoder",
    goal="Geocode a location in to a latitude and longitude, using a web API.",
    backstory="""You have perfected the ability to geocode a location in to a latitude and longitude, using a web API.""",
    tools=[scrape_website_tool],
    verbose=verbose_agents,
    llm="openai/o4-mini",
)

events_researcher = Agent(
    role="Events Researcher",
    goal="Research and document information about local events from websites",
    backstory="""You are a seasoned researcher who specialises in local events, and can extract details about local events from websites.""",
    tools=[scrape_website_tool],
    verbose=verbose_agents,
    llm="openai/o3-mini",
)

weather_forecaster = Agent(
    role="Weather Forecaster",
    goal="Produce a written description about the weather from a web API",
    backstory="""You are an expert Weather Forecaster who produces written descriptions of weather, very much like TV Weather Presenters,
                 using well understood meteorological adjectives; and you can extract details about local weather from a web API.
                 The details you can extract from the web API, and use in your weather forecasting and presentation, are the temperature, rain, wind speed and cloud cover.""",
    tools=[scrape_website_tool],
    verbose=verbose_agents,
    llm="openai/gpt-4o",
)

article_writer = Agent(
    role="Article Writer",
    goal="Craft written articles, that advertise and promote upcoming events",
    backstory="""You are an leading writer who writes for magazines, blogs and advertising.
                 You excel at gathering and presenting information clearly, concisely and in a friendly and polite manner.""",
    verbose=verbose_agents,
    llm="openai/gpt-4.1",
)

artist = Agent(
    role="Promotional Artist",
    goal="Draw images to promote upcoming events",
    backstory="""You are talented artist with a passion for producing fun, light-hearted, friendly and inviting promotional imagery that is suitable for people of all ages.""",
    tools=[DallETool()],
    verbose=verbose_agents,
)


# ────────────────────────────────────────────────────────────────────────────────
# Define the tasks
# ────────────────────────────────────────────────────────────────────────────────
geocode_location = Task(
    description=f"""Retrieve the latitude and longitude for the specified location from the specified Geocoding URL.
                    You will need to pass the specified location in-place of the {{location}} placeholder in the specified Geocoding URL, before you call the URL.
    
                    Location: {location}
                    Geocoding URL: {geocoding_url}

                    From the results of calling the Geocoding URL, get the latitude and longitude from the first item in the United Kingdom.
                    """,
    expected_output="A JSON object containing the latitude, longitude, country, and the location.",
    agent=geocoder,
)

research_events = Task(
    description=f"""Extract the details about local events in the specified location from the specified website URL.
                    You will need to pass the specified location in-place of the {{location}} placeholder in the specified website URL, before you call the URL.
                    
                    Location: {location}
                    Website: {local_events_url}
                    
                    Choose 3 events, with a preference to events that are closer to today.
                    Write a short promotional paragraph about each event, using a professional and friendly tone.
                    """,
    expected_output="Paragraphs of plain text describing the local events.",
    agent=events_researcher,
)

forecast_weather = Task(
    description=f"""Write an engaging paragraph about the local weather in the specified location from the specified web API.
    
                    Location: {location}
                    Web API: {weather_url}
                    
                    You will need to pass the actual latitude and longitude in-place of the {{latitude}} and {{longitude}}
                    placeholders, respectively, in the specified web API URL, before you call the URL.
                    You must get the actual latitude and longitude from the Geocoder.
                    """,
    expected_output="A paragraph of plain text describing the weather forecast.",
    agent=weather_forecaster,
    context=[geocode_location],
)

write_article = Task(
    description=f"""Write a short, engaging, fun and inviting article about local events and weather in {location}.
                    The article muts be based only on the Events Researcher's and Weather Forecaster's summaries.
                    """,
    expected_output="A short article of plain text.",
    agent=article_writer,
    context=[research_events, forecast_weather],
)

draw = Task(
    description=f"""Use the Article Writer's article as the description of the image that you should draw.
                    Produce 3 fun and inviting images about local events and weather in {location},
                    that is suitable for people of all ages, based on the Article Writer's article.
                    Include some famous landmarks from {location}.
                    Each image must be of a different event from the article.
                    The first image should be in cartoon style, the second image should be in a photorealistic style,
                    and the third image should be in the style of David Hockney.
                    You must only use the Article Writer's article as the source of information for the images.
                    """,
    expected_output=f"""Markdown text with two sections.
    
                        The first section should be titled "Visit {location}" and contain the Article Writer's original, unchanged article.
                        
                        The second section should be titled "Images", and contain links to each of the images.
                        The text for each of the links must be a description of what the image is of, and also include the style used.""",
    agent=artist,
    context=[write_article],
)


# ────────────────────────────────────────────────────────────────────────────────
# Demo run
# ────────────────────────────────────────────────────────────────────────────────
result = Crew(
    agents=[
        geocoder,
        events_researcher,
        weather_forecaster,
        article_writer,
        artist,
    ],
    tasks=[
        geocode_location,        
        research_events,
        forecast_weather,
        write_article,
        draw,
    ],
).kickoff()

print(result)
